In [1]:
import pandas as pd
import numpy as np
from math import log2

In [2]:
# Functions

def Gini(Ls, WW = None):
    
    
    if WW == None:
        L = len(Ls)
        WW = [1]*L
    
    W = {}
    Sum = 0
    for x in range(len(Ls)):
        
        if Ls[x] not in W:
            W[Ls[x]] = 0
            
        W[Ls[x]] += WW[x]
        Sum += WW[x]
    
    S = 0
    for x in W:
        S += (W[x]/Sum)**2
        
    return 1 - S

#################################################################


def majority(Ls, WW = None):
    
    
  
    if WW == None:
        L = len(Ls)
        WW = [1]*L
    
    W = {}
    for x in range(len(Ls)):
        
        if Ls[x] not in W:
            W[Ls[x]] = 0
            
        W[Ls[x]] += WW[x]
    
    Max = -1
    majority = None    
    for y in W:
        if W[y] > Max:
            Max = W[y]
            majority = y        
    return 1 - Max/sum(WW)

#################################################################

def entropy(Ls, WW = None):
    
    n = len(Ls)
    if WW == None:
        WW = [1]*n
            
    W = {}
    Sum = 0
    for x in range(n):
        if Ls[x] not in W:
            W[Ls[x]] = 0
        
        W[Ls[x]] += WW[x]
            
        Sum += WW[x]
        
    S = 0
    for x in W:
        S += (W[x]/Sum) * log2(Sum / W[x])

    return S


#################################################################

def Majority(Ls, WW = None, ignor = None):
    
    
    if WW == None:
        L = len(Ls)
        WW = [1]*L
    
    W = {}
    for x in range(len(Ls)):
        
        if Ls[x] not in W:
            W[Ls[x]] = 0
        
        if ignor is None:
            W[Ls[x]] += WW[x]
        else:
            if Ls[x]!= ignor:
                W[Ls[x]] += WW[x]
                
    
    Max = -1
    majority = None    
    for y in W:
        if W[y] > Max:
            Max = W[y]
            majority = y        
    return(majority, len(W))



#################################################################


def Entropy_given_attribute(T, L, a, W = None, Entropy_function = entropy):
    
    n = len(L)
    if W == None:
        W = [1]*n
    
    
    split_l = {}
    split_w = {}
    sum_W = sum(W)
    
    for x in range(n):
        
        u = T[x][a]
        if u not in split_w:
            
            split_w[u] =[]
            split_l[u] = []
            
        split_w[u].append(W[x])
        split_l[u].append(L[x])  
        
    En = 0        
    for x in split_w:
        
        En += sum(split_w[x]) * Entropy_function(split_l[x], split_w[x]) / sum_W
        
    return(En, list(split_w.keys()))    


#################################################################


class DT(object):
    def __init__(self, train, L, attss, depth = -1, WW = None, 
                 Entropy_function = entropy):
        
        self.Entropy_function = Entropy_function
        self.leaf = False 
        self.label, n_values = Majority(L, WW) 
        
        if len(attss) == 0 or n_values == 1 or depth == 0:
            
            self.leaf = True  
            return
        
        self.at_splt, values = self.best_att(train, L, attss, WW)
        
        train_s, lables_s, weight_s = split(train, L, self.at_splt, WW) 
        self.Tree = {}
        
        attss.remove(self.at_splt)
        
        for v in train_s:
               
            self.Tree[v] = DT(train_s[v], lables_s[v], attss, depth - 1, 
                                        weight_s[v], Entropy_function)

        attss.append(self.at_splt)
            
    
    def predict(self, instance):
        
        if self.leaf:
            return self.label
        
        if instance[self.at_splt] in self.Tree:
            return self.Tree[instance[self.at_splt]].predict(instance)   
        
        return self.label   
    
    
    def best_att(self, train, L, attss, WW):
        
        lable_Ent = entropy(L, WW)
        Max = -1
        Best = None
        Best_values = None
    
        for attribute in attss: 
            temp, temp_values = Entropy_given_attribute(train, L, 
                                                        attribute, WW, self.Entropy_function) 
            if lable_Ent - temp >  Max:
                Max = lable_Ent - temp
                Best = attribute
                Best_values = temp_values
                    
        return(Best, Best_values)

#################################################################    
    
def split(train, label, attribute, WW = None):
    
    n = len(label)
    if WW == None:
        WW = [1]*n
    
    split_w = {}
    split_t = {}
    split_l = {}
    
    for x in range(len(label)):
        
        #print('x = ', x)
        #print('attribute = ', attribute)
        txa = train[x][attribute]
        if txa not in split_t:
            
            split_w[txa] = []
            split_t[txa] = []
            split_l[txa] = []
            
        split_w[txa].append(WW[x])
        split_t[txa].append(train[x])
        split_l[txa].append(label[x])
        
    return (split_t, split_l, split_w)

In [3]:
# txtfile = open('bank/data-desc.txt', 'r')
# print(txtfile.read())

In [4]:
C = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 
 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
types = ['numeric', 'categorical', 'categorical', 'categorical', 'binary', 'numeric', 
                      'binary', 'binary', 'categorical', 'numeric', 'categorical', 'numeric', 
                      'numeric', 'numeric', 'numeric', 'categorical', 'binary']
dic= dict(zip(C, types))

In [5]:
train = pd.read_csv('bank/train.csv', names = C)
test = pd.read_csv('bank/test.csv', names = C)
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,41,services,married,secondary,no,0,yes,no,unknown,5,may,114,2,-1,0,unknown,no
1,48,blue-collar,single,secondary,no,312,yes,yes,cellular,3,feb,369,2,-1,0,unknown,no
2,55,technician,married,secondary,no,1938,no,yes,cellular,18,aug,193,1,386,3,success,yes
3,54,admin.,married,tertiary,no,59,yes,no,cellular,10,jul,268,1,-1,0,unknown,no
4,34,management,single,tertiary,no,2646,no,no,cellular,14,apr,142,1,-1,0,unknown,yes


In [6]:
median_dict = {}
Train_new =pd.DataFrame()
Test_new =pd.DataFrame()
for name in C:
    if dic[name] == 'numeric':
        M = train[name].median()
        median_dict[name] = M
        Train_new[name+ '>' + str(M)] = np.where(train[name]  > M, "yes", 'no')
        Test_new[name+ '>' + str(M)] = np.where(test[name]  > M, "yes", 'no')
    else:
        Train_new[name] = train[name]
        Test_new[name] = test[name]

In [7]:
Train = []
Label = []
for i in range(len(Train_new)):
    temp = list(Train_new.loc[i])
    Train.append(temp[:-1])
    Label.append(temp[-1])

In [8]:
Test = []
Test_Label = []
for i in range(len(Test_new)):
    temp = list(Test_new.loc[i])
    Test.append(temp[:-1])
    Test_Label.append(temp[-1])

In [9]:
attributes = list(range(len(C)-1))

In [10]:
attributes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [11]:
def prediction(Model, data, label):
    pred = []
    for i in range(len(data)):
        temp = Model.predict(data[i])
        pred.append(temp)
    return (np.array(label) == np.array(pred)).mean()

In [12]:
for f in [entropy, majority, Gini]:
    print("the method is baces on ", f, "index")
    for depth in range(1,16):
        Model = DT(Train, Label, attributes, depth = depth, Entropy_function = f)
        print("for depth = {}, error for training is {}".format(depth, 1-prediction(Model, Train, Label)))

the method is baces on  <function entropy at 0x7f9851800310> index
for depth = 1, error for training is 0.11919999999999997
for depth = 2, error for training is 0.10599999999999998
for depth = 3, error for training is 0.10060000000000002
for depth = 4, error for training is 0.07999999999999996
for depth = 5, error for training is 0.06240000000000001
for depth = 6, error for training is 0.047799999999999954
for depth = 7, error for training is 0.03739999999999999
for depth = 8, error for training is 0.029000000000000026
for depth = 9, error for training is 0.022399999999999975
for depth = 10, error for training is 0.018000000000000016
for depth = 11, error for training is 0.015000000000000013
for depth = 12, error for training is 0.013800000000000034
for depth = 13, error for training is 0.013599999999999945
for depth = 14, error for training is 0.01319999999999999
for depth = 15, error for training is 0.01319999999999999
the method is baces on  <function majority at 0x7f9851800160> ind

In [13]:
for f in [entropy, majority, Gini]:
    print("the method is baces on ", f, "index")
    for depth in range(1,16):
        Model = DT(Train, Label, attributes, depth = depth, Entropy_function = f)
        print("for depth = {}, error for test is {}".format(depth, 1-prediction(Model, Test, Test_Label)))

the method is baces on  <function entropy at 0x7f9851800310> index
for depth = 1, error for test is 0.12480000000000002
for depth = 2, error for test is 0.11140000000000005
for depth = 3, error for test is 0.10699999999999998
for depth = 4, error for test is 0.11480000000000001
for depth = 5, error for test is 0.12560000000000004
for depth = 6, error for test is 0.13280000000000003
for depth = 7, error for test is 0.139
for depth = 8, error for test is 0.14459999999999995
for depth = 9, error for test is 0.14700000000000002
for depth = 10, error for test is 0.1532
for depth = 11, error for test is 0.15500000000000003
for depth = 12, error for test is 0.15639999999999998
for depth = 13, error for test is 0.15680000000000005
for depth = 14, error for test is 0.15680000000000005
for depth = 15, error for test is 0.15639999999999998
the method is baces on  <function majority at 0x7f9851800160> index
for depth = 1, error for test is 0.11660000000000004
for depth = 2, error for test is 0.108